<a href="https://colab.research.google.com/github/BenTicali/CS167_Notes/blob/main/Day05_Notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 05 Notes: 
There are 3 parts to today's code:
1. subsetting review
2. kNN implementation
3. Normalization

## Part 1: Subsetting review:

In [12]:
import pandas
import numpy
from google.colab import drive
drive.mount('/content/drive')
#go through authentication step

#import the data and take a look at it
data = pandas.read_csv('/content/drive/MyDrive/CS167/datasets/irisData.csv')
data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
#some subsetting practice to help with Notebook 1
#Grab the rows where sepal length >4 and petal length <4
subset_rows = data[(data['sepal length']>4)&(data['petal length']<4)]
subset_rows.head()

,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
#grab a subset of columns
subset_cols = data[['sepal length', 'sepal width', 'species']]
subset_cols.head()

,sepal length,sepal width,species
0,5.1,3.5,Iris-setosa
1,4.9,3.0,Iris-setosa
2,4.7,3.2,Iris-setosa
3,4.6,3.1,Iris-setosa
4,5.0,3.6,Iris-setosa


In [5]:
#subset both rows and columns in 2 steps:
subset_rows = data[(data['sepal length']>4)&(data['petal length']<4)] #grab the rows we want
subset_rows_cols_2step = subset_rows[['sepal length', 'sepal width', 'species']] #from those rows, grab the columns we want
subset_rows_cols_2step.head() 

,sepal length,sepal width,species
0,5.1,3.5,Iris-setosa
1,4.9,3.0,Iris-setosa
2,4.7,3.2,Iris-setosa
3,4.6,3.1,Iris-setosa
4,5.0,3.6,Iris-setosa


In [6]:
#grab a subset of rows and columns at the same time. 
subset_rows_cols_1step =  data[(data['sepal length']>4)&(data['petal length']<4)][['sepal length', 'sepal width', 'species']]
subset_rows_cols_1step.head()

,sepal length,sepal width,species
0,5.1,3.5,Iris-setosa
1,4.9,3.0,Iris-setosa
2,4.7,3.2,Iris-setosa
3,4.6,3.1,Iris-setosa
4,5.0,3.6,Iris-setosa


## Part 2: kNN Implementation
We went over this step by step in class on Thursday, but here is what the function looks like put together.

In [9]:
def knn(specimen, data, k):
  #add a column for the distance:
  data['distance_to_new'] = numpy.sqrt(
    (specimen['petal length']-data['petal length'])**2
    +(specimen['sepal length']-data['sepal length'])**2
    +(specimen['petal width']-data['petal width'])**2
    +(specimen['sepal width']-data['sepal width'])**2)
  
  #sort the values:
  sorted_data = data.sort_values(['distance_to_new']) 

  #return the most commonly occuring class of the k closest neighbors.
  return sorted_data.iloc[0:k]['species'].mode()[0]


In [13]:
# dictionary of values
iris = {}
iris['petal length'] = 5.1
iris['sepal length'] = 7.2
iris['petal width'] = 1.5
iris['sepal width'] = 2.5
# 5-NN 
knn(iris,data,5)

'Iris-versicolor'

In [37]:
# weighted kNN:

def w_knn(specimen, data, k):
  #calculate the distance
  data['distance_to_new'] = numpy.sqrt((specimen['petal length']-data['petal length'])**2
                                       +(specimen['sepal length']-data['sepal length'])**2
                                       +(specimen['petal width']-data['petal width'])**2
                                       +(specimen['sepal width']-data['sepal width'])**2)
  #calculate the weights (remember, the weights are 1/d^2)
  data['weight'] = 1/data['distance_to_new']**2
  #sort the data so that the closest neighbors are first
  subset_data = data.sort_values(['distance_to_new'])
  #find the closest k
  subset_data = subset_data.head(k)
  # use groupby to sum the weights of each species in the closest k 
  weight_sum = subset_data.groupby(['species'])['weight'].sum()
  # return the class that has the larger sum of weight
  return weight_sum.idxmax()

In [38]:
# dictionary of values
iris = {}
iris['petal length'] = 5.1
iris['sepal length'] = 7.2
iris['petal width'] = 1.5
iris['sepal width'] = 2.5
# 5-NN 
w_knn(iris,data,5)

'Iris-versicolor'

## Part 3: Normalization

In [ ]:
# Normalizing columns (z-score)
titanic = pandas.read_csv('/content/drive/MyDrive/CS167/datasets/titanic.csv')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
#convert sex to numeric
titanic['sex'] = titanic['sex'].replace(to_replace='female', value=1)
titanic['sex'] = titanic['sex'].replace(to_replace='male', value=0)
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,0,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,1,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,1,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,1,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,0,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
#calculate mean and std
s_mean = titanic['sex'].mean()
s_std = titanic['sex'].std()

#replace column with each entry's z-score
titanic['sex'] = (titanic["sex"] - s_mean)/s_std

In [ ]:
#age
#calculate mean and std
a_mean = titanic['age'].mean()
a_std = titanic['age'].std()

#replace column with each entry's z-score
titanic['age'] = (titanic["age"] - a_mean)/a_std
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,-0.737281,-0.530005,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,1.354813,0.571430,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,1.354813,-0.254646,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,1.354813,0.364911,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,-0.737281,0.364911,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
